# Where Should We Move?

In [37]:
import pandas as pd
import numpy as np
import geopandas as gpd
import branca.colormap as cm
import folium
from time import time
from folium.plugins import FastMarkerCluster, MarkerCluster
from os import listdir
from os.path import isfile, join
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#Check out https://www.city-data.com/

## Calculation flags / variables

In [48]:
import_data = False

### Walkability
* Rural == 1.2
* Suburban == 8.3
* Historic Main Street = 13.7
* City Center == 17.5

In [39]:
walk_output_path = 'Cached/walk_df_agg_by_zip.csv'
def create_tract_from_codes(row):
    state = str(row['STATEFP'])
    county = str(row['COUNTYFP'])
    tract = str(row['TRACTCE'])
    while len(state) < 2:
        state = '0' + state
    while len(county) < 3:
        county = '0' + county
    while len(tract) < 6:
        tract = '0' + tract
    combined = state + county + tract
    return int(combined)

def zip_str(row):
    zip_str = str(int(row['zip']))
    while len(zip_str) < 5:
        zip_str = '0' + zip_str
    return zip_str

# walk_bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
walk_bins = [0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

if import_data:
    tract_to_zip = pd.read_excel('DataSets/TRACT_ZIP_122021.xlsx')
    walk_df = pd.read_csv('DataSets/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')
    walk_df['Tract'] = walk_df.apply(lambda row: create_tract_from_codes(row), axis=1)
    walk_df_with_zip = walk_df.merge(tract_to_zip, how='left', left_on='Tract', right_on='tract')
    walk_df_agg_by_zip = pd.DataFrame(walk_df_with_zip.groupby('zip').agg({'NatWalkInd': [np.mean, np.max, np.min, np.std]}))
    walk_df_agg_by_zip = walk_df_agg_by_zip['NatWalkInd']
    walk_df_agg_by_zip['1stdbelow'] = walk_df_agg_by_zip['mean'] - walk_df_agg_by_zip['std']
    walk_df_agg_by_zip['1stdabove'] = walk_df_agg_by_zip['mean'] + walk_df_agg_by_zip['std']
    walk_df_agg_by_zip['zip'] = walk_df_agg_by_zip.index.astype(int).astype(str)
    walk_df_agg_by_zip = walk_df_agg_by_zip.reset_index(drop=True)
    walk_df_agg_by_zip['walk_score'] = pd.cut(walk_df_agg_by_zip['mean'], walk_bins, labels = [i for i in range(1, len(walk_bins))]).astype(int)
    walk_df_agg_by_zip.columns = ['mean', 'max', 'min', 'std', '1std_below', '1std_above', 'zip', 'walk_score']
    walk_df_agg_by_zip['zip'] = walk_df_agg_by_zip.apply(lambda row: zip_str(row), axis=1)
    walk_df_agg_by_zip.to_csv(walk_output_path)
else:
    walk_df_agg_by_zip = pd.read_csv(walk_output_path).drop('Unnamed: 0', axis=1)
walk_df_agg_by_zip['zip'] = walk_df_agg_by_zip.apply(lambda row: zip_str(row), axis=1)
walk_df_agg_by_zip.head(2)

,mean,max,min,std,1std_below,1std_above,zip,walk_score
0,8.25000,9.83333,6.66667,2.23917,6.01083,10.48917,00501,1
1,4.79167,7.33333,2.66667,1.70841,3.08326,6.50007,00601,1


### Zillow House Value Index

In [59]:
zhvi_output_path = 'Cached/zillow_sfh_zip.csv'
def get_recent_zhvi(row):
    recent_value = row.iloc[270:].dropna().values[-1]
    try:
        return recent_value
    except:
        return None

zhvi_bins = [0, 350000, 400000, 500000, 600000, 700000, 800000, 10000000]

if import_data:
#     zip_to_lat = pd.read_csv('DataSets/ziptolat.csv')
#     zip_to_lat['zip'] = zip_to_lat['ZIP']
#     zip_to_lat.drop('ZIP', axis=1, inplace=True)
    
    #3br by zip
    zillow_sfh_zip = pd.read_csv('DataSets\Zillow\Zip_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
#     zillow_sfh_zip = zillow_sfh_zip.merge(zip_to_lat, how='inner', left_on='RegionName', right_on='zip')
    zillow_sfh_zip["ZHVI"] = zillow_sfh_zip.apply(lambda row: get_recent_zhvi(row), axis=1)
    zillow_sfh_zip['zip'] = zillow_sfh_zip['RegionName']
    zillow_sfh_zip.drop('RegionName', axis=1, inplace=True)

    start = time()
    
#     if filter_data:  
#         zillow_sfh_zip = zillow_sfh_zip[zillow_sfh_zip["State"] == target_state.upper()]
#     zillow_sfh_zip = zillow_sfh_zip[['ZHVI', 'Metro', 'City', 'zip', 'State']]
        
#     if cost_cutoff:
#         sfh_filtered = sfh_filtered[sfh_filtered['ZHVI'] <= max_price]
    
    #Creating price scores to help standardize analysis across geos and minimize impact out outliers
    zillow_sfh_zip['zhvi_score'] = pd.cut(zillow_sfh_zip['ZHVI'], zhvi_bins, labels = [len(zhvi_bins) - i for i in range(1, len(zhvi_bins))]).astype(int)
    zillow_sfh_zip.to_csv(zhvi_output_path)
    
    end = time()
    print(end-start)
else:
    zillow_sfh_zip = pd.read_csv(zhvi_output_path).drop('Unnamed: 0', axis=1)
zillow_sfh_zip['zip'] = zillow_sfh_zip.apply(lambda row: zip_str(row), axis=1)
zillow_sfh_zip = zillow_sfh_zip[['zip', 'StateName', 'State', 'City', 'Metro', 'CountyName', 'ZHVI', 'zhvi_score']]
zillow_sfh_zip.head(2)

,zip,StateName,State,City,Metro,CountyName,ZHVI,zhvi_score
0,10025,NY,NY,New York,New York-Newark-Jersey City,New York County,1987683.00000,1
1,60657,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,659740.00000,3


### Weather data

In [60]:
#Weather data comes from pre-cooked files in DataSets/Weather folder
annual_weather_data_output_path = 'Cached/annual_weather_data.csv'

weather_bins = [0, 60, 65, 70, 75, 80, 100] 

if import_data:
    annual_weather_data = pd.DataFrame()
    annual_path = 'DataSets/Weather/Annual/'

    for file in listdir(annual_path):
        zip_code = file[:5]
        annual_data = pd.read_csv(annual_path + file, index_col=0)       
        annual_flat = pd.DataFrame(annual_data.values.flatten()).T
        annual_flat.columns = ['THI_score_mean', 'tempmin_score_mean', 'over90_score_mean', 'rainy_sum_score_mean', 'cloud_score_mean', 'weather_index_mean',
                           'THI_score_min', 'tempmin_score_min', 'over90_score_min', 'rainy_sum_score_min', 'cloud_score_min', 'weather_index_min',
                            'THI_score_max', 'tempmin_score_max', 'over90_score_max', 'rainy_sum_score_max', 'cloud_score_max', 'weather_index_max']
        annual_flat['zip'] = zip_code
        annual_weather_data = annual_weather_data.append(annual_flat)
        annual_weather_data['weather_score'] = pd.cut(annual_weather_data['weather_index_mean'], weather_bins, labels = [i for i in range(1, len(weather_bins))]).astype(int)
    
    annual_weather_data.to_csv(annual_weather_data_output_path)
else:
    annual_weather_data = pd.read_csv(annual_weather_data_output_path).drop('Unnamed: 0', axis=1)
annual_weather_data['zip'] = annual_weather_data.apply(lambda row: zip_str(row), axis=1)
annual_weather_data.head()

,THI_score_mean,tempmin_score_mean,over90_score_mean,rainy_sum_score_mean,cloud_score_mean,weather_index_mean,THI_score_min,tempmin_score_min,over90_score_min,rainy_sum_score_min,cloud_score_min,weather_index_min,THI_score_max,tempmin_score_max,over90_score_max,rainy_sum_score_max,cloud_score_max,weather_index_max,zip,weather_score
0,3.00000,4.00000,5.91667,5.00000,4.16667,68.05556,3.00000,4.00000,5.66667,5.00000,3.66667,65.22222,3.00000,4.00000,6.00000,5.00000,5.00000,71.66667,00601,3
1,3.00000,4.00000,5.91667,5.00000,4.16667,68.05556,3.00000,4.00000,5.66667,5.00000,3.66667,65.22222,3.00000,4.00000,6.00000,5.00000,5.00000,71.66667,00602,3
2,3.00000,4.00000,5.91667,5.00000,4.16667,68.05556,3.00000,4.00000,5.66667,5.00000,3.66667,65.22222,3.00000,4.00000,6.00000,5.00000,5.00000,71.66667,00603,3
3,3.00000,4.00000,5.91667,5.00000,4.16667,68.05556,3.00000,4.00000,5.66667,5.00000,3.66667,65.22222,3.00000,4.00000,6.00000,5.00000,5.00000,71.66667,00606,3
4,3.00000,4.00000,5.91667,5.00000,4.16667,68.05556,3.00000,4.00000,5.66667,5.00000,3.66667,65.22222,3.00000,4.00000,6.00000,5.00000,5.00000,71.66667,00610,3


## Aggregating

In [61]:
#Importing zipcode, lat, lng for all zipcodes in the united states
ziplatlon = pd.read_csv('DataSets/ziptolat.csv')
ziplatlon['zip'] = ziplatlon['ZIP']
ziplatlon['zip'] = ziplatlon.apply(lambda row: zip_str(row), axis = 1)
ziplatlon.drop('ZIP', axis=1, inplace=True)
ziplatlon.head(2)

,LAT,LNG,zip
0,18.18055,-66.74996,00601
1,18.36194,-67.17560,00602


In [69]:
combined = ziplatlon.merge(walk_df_agg_by_zip, how='left', on='zip')
combined = combined.merge(zillow_sfh_zip, how='left', on='zip')
combined = combined.merge(annual_weather_data, how='left', on='zip')
combined['location'] = combined['zip'] + ' ' + combined['Metro'] + ' ' + combined['City'] + ' ' + combined['State']
combined.head(2)

,LAT,LNG,zip,mean,max,min,std,1std_below,1std_above,walk_score,StateName,State,City,Metro,CountyName,ZHVI,zhvi_score,THI_score_mean,tempmin_score_mean,over90_score_mean,rainy_sum_score_mean,cloud_score_mean,weather_index_mean,THI_score_min,tempmin_score_min,over90_score_min,rainy_sum_score_min,cloud_score_min,weather_index_min,THI_score_max,tempmin_score_max,over90_score_max,rainy_sum_score_max,cloud_score_max,weather_index_max,weather_score,location
0,18.18055,-66.74996,00601,4.79167,7.33333,2.66667,1.70841,3.08326,6.50007,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00000,4.00000,5.91667,5.00000,4.16667,68.05556,3.00000,4.00000,5.66667,5.00000,3.66667,65.22222,3.00000,4.00000,6.00000,5.00000,5.00000,71.66667,3,NaN
1,18.36194,-67.17560,00602,4.54545,7.33333,2.66667,1.43196,3.11350,5.97741,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00000,4.00000,5.91667,5.00000,4.16667,68.05556,3.00000,4.00000,5.66667,5.00000,3.66667,65.22222,3.00000,4.00000,6.00000,5.00000,5.00000,71.66667,3,NaN


## Calculating ReddingScore

In [81]:
threshold = False
if threshold:
    walk_min = 5 #out of 10
    zhvi_min = 3 #out of 10
    weather_min = 3 #out of 100

    combined_filtered = combined[combined['walk_score'] >= walk_min]
    combined_filtered = combined[combined['zhvi_score'] >= zhvi_min]
    combined_filtered = combined[combined['weather_index_mean'] >= weather_min]
else:
    combined_filtered = combined

walk_weight = 35
zhvi_weight = 30
weather_weight = 35

max_walk_score = len(walk_bins) - 1
max_zhvi_score = len(zhvi_bins) - 1
max_weather_score = len(weather_bins) - 1

combined_filtered['ReddingScore'] = (combined_filtered['walk_score'] / max_walk_score * walk_weight) \
                                   + (combined_filtered['zhvi_score'] / max_zhvi_score * zhvi_weight) \
                                   + (combined_filtered['weather_score'] / max_weather_score * weather_weight) 

In [82]:
print('{} out of {} zip codes have all necessary data'.format(combined_filtered[combined_filtered['ReddingScore'].notna()].shape[0], combined_filtered.shape[0]))

23262 out of 33144 zip codes have all necessary data


In [83]:
combined_filtered = combined_filtered[combined_filtered['ReddingScore'].notna()]

## Building Map

In [ ]:
target_state_name = "california"
create_markers = True

target_state_name = target_state_name.lower()
state_name_dict = pd.read_csv('DataSets/state_names.csv')
target_state = state_name_dict[state_name_dict['State'].str.lower() == target_state_name.lower()]['Postal'].values[0].lower()

geojson_path = 'DataSets/geo_json/' + target_state + '_' + target_state_name + '_zip_codes_geo.min.json'
geojson_df = gpd.read_file(geojson_path)[['ZCTA5CE10', 'geometry']]
geojson_df.columns = ['zip', 'geometry']

combined_with_geo = geojson_df.merge(combined_filtered, how='inner', on='zip')

x_map = combined_with_geo['LAT'].mean()
y_map = combined_with_geo['LNG'].mean()
m = folium.Map(location=[x_map, y_map], zoom_start=7, tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

# ReddingScoreScale = np.quantile([0, 100], (0, .50, 0.60, .70, 0.75, .80, 0.85, .90, 1)).tolist()
ReddingScoreScale = np.quantile([0, 100], (0, .70, .725, 0.75, .775, .80, .825, 0.85, .875, 1)).tolist()

folium.Choropleth(
     geo_data=combined_with_geo,
     name='ReddingScore',
     data=combined_with_geo,
     columns=['zip','ReddingScore'],
     key_on="properties.zip",
     fill_color='YlGnBu',
     threshold_scale=ReddingScoreScale,
     fill_opacity=0.7,
     line_opacity=0.2,
     legend_name='ReddingScore',
     nan_fill_color="#fffff0",
     smooth_factor=0
    ).add_to(m)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

# combined_with_geo = combined_with_geo.dropna()
# combined_with_geo['location'] = combined_with_geo['zip'] + ' ' + combined_with_geo['Metro'] + ' ' + combined_with_geo['City'] + ' ' + combined_with_geo['State']
fields_to_show = ['location', 'ReddingScore', 'mean', 'ZHVI', 'weather_index_mean']
field_aliases = ['Location','ReddingScore', 'WalkScore', 'MediumHomePrice', 'WeatherScore']
# for field in fields_to_show[1:]:
#     combined_with_geo[field] = combined_with_geo[field].astype(int)

if create_markers:
    folium.features.GeoJson(
        combined_with_geo,
        style_function = style_function, 
        control = False,
        highlight_function = highlight_function, 
        tooltip = folium.features.GeoJsonTooltip(
            fields = fields_to_show,
            aliases = field_aliases,
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
            )
        ).add_to(m)

m

# Hunting

In [87]:
combined.sort_values('ReddingScore', ascending=False).head(20)[['location', 'ReddingScore', 'weather_index_mean', 'weather_score', 'ZHVI', 'walk_score']]

,location,ReddingScore,weather_index_mean,weather_score,ZHVI,walk_score
27779,79901 El Paso El Paso TX,81.43939,76.41667,5,133324.00000,7.00000
31241,95202 Stockton-Lodi Stockton CA,80.33550,75.33333,5,382511.00000,8.00000
27781,79903 El Paso El Paso TX,78.25758,76.41667,5,162355.00000,6.00000
29539,87108 Albuquerque Albuquerque NM,78.25758,77.56944,5,265162.00000,6.00000
30928,93728 Fresno Fresno CA,78.25758,75.04861,5,300572.00000,6.00000
30922,93721 Fresno Fresno CA,78.25758,75.04861,5,307043.00000,6.00000
29098,85034 Phoenix-Mesa-Scottsdale Phoenix AZ,78.25758,75.43056,5,284268.00000,6.00000
29109,85051 Phoenix-Mesa-Scottsdale Phoenix AZ,78.25758,76.05556,5,342492.00000,6.00000
30896,93650 Fresno Fresno CA,78.25758,75.04861,5,298221.00000,6.00000
29541,87110 Albuquerque Albuquerque NM,78.25758,77.56944,5,285078.00000,6.00000
